In [ ]:
# AIM: 

# >>> Perform MaxPooling at RF=5
# >>> GAP
# >>> Add rotation, we guess that 5-7 degrees should be sufficient. 
# >>> Add LR Scheduler

In [ ]:
### Libraries

from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
## transforms with Augmentations

train_transforms = transforms.Compose([
    transforms.RandomRotation((-5.0, 5.0), fill=(1,)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:
## dataset split

train = datasets.MNIST("./data", train=True, download=True, transform=train_transforms)
test = datasets.MNIST("./data", train=False, download=True, transform=test_transforms)

In [ ]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

In [ ]:
## Neural Network

dropout_value = 0.075
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # INPUT-BLOCK
        self.inputblock = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3,3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # n_out = 28 - 3 + 1 = 26, # Jout = 1 # Rfout = 1 + (3-1)*1 = 3

        # BLOCK -- [1]
        self.hiddenblock1 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # n_out = 26 - 3 + 1 = 24, # Jout = 1, # Rfout = 3 + (3-1)*1 = 5

        self.transistionblock = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=14, kernel_size=(1,1))
        ) # n_out = 24, # Jout = 1, Rfout = 5
        

        ## TRANSISTION BLOCK
        self.maxPool1 = nn.MaxPool2d((2, 2)) 
        # n_out = (24-2)/2 + 1 = 12, # Jout = 1*2 = 2, # Rfout = 7 + (2-1)*1 = 8
        
        # BLOCK -- [2]
        self.hiddenblock2 = nn.Sequential(
            nn.Conv2d(in_channels=14, out_channels=14, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(14),
            nn.Dropout(dropout_value)
        ) # n_out = 12 - 3 + 1 = 10, # Jout = 1, # Rfout = 5 + (3-1)*1 = 7

        # BLOCK -- [3]
        self.hiddenblock3 = nn.Sequential(
            nn.Conv2d(in_channels=14, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # n_out = (10-3+1) = 8, # Jout = 2, # Rfout = 8 + (3-1)*2 = 12


        # BLOCK --[5]
        self.hiddenblock5 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(dropout_value)
        ) # n_out = 8 - 3 + 1 = 6, # Jout = 2, # Rfout = 12 + (3-1)*2 = 16

        self.hiddenblock6 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3,3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(dropout_value)
        ) # n_out = 6 - 3 + 1 = 4, # Jout = 2, # Rfout = 16 + (3-1)*2 = 20


        self.gapblock = nn.Sequential(
            nn.AvgPool2d(kernel_size=(4, 4))) # n_out = 5 - 5 + 1=1, # Jout = 2, # Rfout = 24 + (5-1)*2 = 32

        self.outputblock = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(1,1), padding=0, bias=False)
        )

    def forward(self, x):
        x = self.inputblock(x)
        x = self.hiddenblock1(x)
        x = self.transistionblock(x)
        x = self.maxPool1(x)
        x = self.hiddenblock2(x)
        x = self.hiddenblock3(x)
        x = self.hiddenblock5(x)
        x = self.hiddenblock6(x)
        x = self.gapblock(x)
        # x = self.outputblock(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

device = 'cuda'
model = Net().to(device)
        


In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

summary(model, input_size=(1, 28, 28))

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
from torch.optim.lr_scheduler import StepLR

model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)